In [269]:
import pandas as pd

names=pd.read_csv('/Users/hubertcheung/Downloads/temp.csv')
names=names.dropna()
nameparse=names['name'].str.split()

nameparse = nameparse[nameparse.apply(lambda x: (isinstance(x, list) and len(x)>0))]
nameparse = nameparse[nameparse.apply(lambda x: 'LTD' not in x)]

In [270]:
with open('canton.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

canton = [line.strip() for line in lines]

In [271]:
with open('pinyin.txt','r',encoding='utf-8') as file:
    lines=file.readlines()

pinyin = [line.strip() for line in lines]

In [272]:
with open('last_name_pinyin.txt','r',encoding='utf-8') as file:
    lines=file.readlines()
    
last_name_pinyin = [line.strip() for line in lines]

In [273]:
with open('last_name_canton.txt','r',encoding='utf-8') as file:
    lines=file.readlines()
    
last_name_canton = [line.strip() for line in lines]

In [274]:
df = pd.DataFrame(nameparse)

In [275]:
pinyin_unique = list(set(pinyin) - (set(canton) & set(pinyin)))
canton_unique = list(set(canton) - (set(canton) & set(pinyin)))

In [284]:
#The main part of the identification. 
def identify(name):
    if len(name)==1: return 'unknown'
    if len(name)>=2:
        if name[0] in last_name_pinyin:
            if name[0] not in last_name_canton:
                if name[1] in canton_unique:
                    return 'unknown'
                elif len(name)>=3 and name[2] in canton_unique:
                    return 'unknown'
                else: return 'ML'
            if name[0] in last_name_canton:
                if name[1] in pinyin_unique:
                    return 'ML'
                elif name[1] in canton_unique:
                    return 'HK'
                elif len(name)>=3:
                    if name[2] in pinyin_unique: return 'ML'
                    elif name[2] in canton_unique: return 'HK'
                    elif len(name)>=4:
                        if name[3] in pinyin_unique: return 'ML'
                        if name[3] in canton_unique: return 'HK'
                elif len(name)==2: return 'ML'
                else: return 'HK'
                    
        elif name[0] in last_name_canton: return 'HK'
        else:
            return 'F'

In [277]:
def countname(a):
    count=0
    for item in a:
        if item == '&':
            count+=1
    return count

counting = nameparse.apply(lambda x: countname(x))

In [278]:
def locate(a,n):
    count = 0
    for i,item in enumerate(a):
        if item == '&':
            count += 1
        if count == n:
            return i
    if n == 0:
        return 0
    if n == countname(a):
        return len(a)

In [279]:
def clean(mylist):
    i = 1
    newlist = []
    consecutive = False
    for item in mylist:
        if item == '&':
            if not consecutive:
                newlist.append(item)
                consecutive = True
        else:
            newlist.append(item)
            consecutive = False
    if mylist[-1] == '&':
        mylist = mylist[:-1]
    return newlist

In [280]:
nameparse = nameparse.apply(lambda x: clean(x))

In [281]:
from itertools import chain, combinations

def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r+1) for r in range(len(s)))

outcome = [set(x) for x in powerset({'HK','ML','F','unknown'})]

In [285]:
#This only works on multiple names.
def nameloop(namelist):
    namecount = countname(namelist) #Notice that here namecount is actually the number of names - 1.
    origin = []
    for num in range(namecount+1):
        name = namelist[locate(namelist,num):locate(namelist,num+1)]
        if name[0] == '&':
            name = name[1:]
        origin.append(identify(name))
    origin_set = set(origin)
    
    for item in outcome:
        if item == origin_set:
            return item
    #Directly return origin_set may incur problem about order of elements.

In [286]:
import time
start_time = time.process_time() # store the start time

f = lambda x: identify(x) if countname(x)==0 else None
df['single'] = df['name'].apply(f)
        
g = lambda x: nameloop(x) if countname(x)>0 else None
df['multiple'] = df['name'].apply(g)

end_time = time.process_time() # store the end time
print(f"The code took {end_time - start_time} seconds of CPU time to run.")

df.to_csv('df.csv')

The code took 32.94384600000001 seconds of CPU time to run.


In [ ]:
df['name'][1495]
nameloop_test(df['name'][1495])

In [ ]:
'MOU' in last_name_canton